In [0]:
PROJECT_PATH = '/home/dobos/project/ga_isochrones/python:' + \
    '/home/dobos/project/ga_pfsspec_all/python:' + \
    '/home/dobos/project/pysynphot'

GRID_PATH = '/datascope/subaru/data/pfsspec/models/stellar/grid/phoenix/phoenix_HiRes'
FILTER_PATH = '/datascope/subaru/data/pfsspec/subaru/hsc/filters/fHSC-g.txt'

ARMS = [ 'b', 'r', 'mr', 'n' ]
FIT_ARMS = [ 'b', 'mr', 'n' ]

DETECTOR_PATH = '/datascope/subaru/data/pfsspec/subaru/pfs/arms/{}.json'
DETECTORMAP_PATH = '/datascope/subaru/data/pfsspec/drp_pfs_data/detectorMap/detectorMap-sim-{}1.fits'
PSF_PATH = '/datascope/subaru/data/pfsspec/subaru/pfs/psf/import/{}.2'
SKY_PATH = '/datascope/subaru/data/pfsspec/subaru/pfs/noise/import/sky.see/{}/sky.h5'
MOON_PATH = '/datascope/subaru/data/pfsspec/subaru/pfs/noise/import/moon/{}/moon.h5'

In [0]:
import os
import sys
import logging
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
from scipy.interpolate import interp1d
import h5py as h5
from tqdm.notebook import trange, tqdm
from collections.abc import Iterable
import pickle

In [0]:
plt.rc('font', size=7)

In [0]:
# Allow load project as module
for p in reversed(PROJECT_PATH.split(':')):
    sys.path.insert(0, p)

os.environ['PYTHONPATH'] = PROJECT_PATH.split(':')[0]

In [0]:
os.environ['PYTHONPATH']

In [0]:
# if 'debugpy' not in sys.modules:
#     import debugpy
#     debugpy.listen(("localhost", 5683))

In [0]:
%load_ext autoreload

In [0]:
%autoreload 2

# Radial velocity fit with different methods

## Load model grid

In [0]:
from pfs.ga.pfsspec.core.grid import ArrayGrid
from pfs.ga.pfsspec.stellar.grid import ModelGrid
from pfs.ga.pfsspec.stellar.grid.bosz import Bosz
from pfs.ga.pfsspec.stellar.grid.phoenix import Phoenix

In [0]:
fn = os.path.join(GRID_PATH, 'spectra.h5')
grid = ModelGrid(Phoenix(), ArrayGrid)
grid.preload_arrays = False
grid.load(fn, format='h5')

In [0]:
model = grid.get_model(M_H=-0.5, T_eff=5500, log_g=4.5, a_M=0.0)

# model = grid.get_model(M_H=-0.5, T_eff=6000, log_g=4.5, a_M=0.0)

model.M_H, model.T_eff, model.log_g, model.a_M

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

ax.plot(model.wave, model.flux, lw=0.3)

## Load detector PSF

In [0]:
from pfs.ga.pfsspec.core.obsmod.psf import GaussPsf, PcaPsf

In [0]:
gauss_psf = {}
pca_psf = {}
template_psf = {}

for arm in ARMS:
    gauss_psf[arm] = GaussPsf()
    gauss_psf[arm].load(os.path.join(PSF_PATH.format(arm), 'gauss.h5'))

    # print(f'mean pixel size for arm {arm}', np.diff(detector[arm].get_wave()[0]).mean())
    # print(f'mean sigma and FWHM for arm {arm}', gauss_psf[arm].sigma.mean(), 2.355 * gauss_psf[arm].sigma.mean())

    s = gauss_psf[arm].get_optimal_size(grid.wave)
    # print(f'optimal kernel size for arm {arm}:', s)

    pca_psf[arm] = PcaPsf()
    pca_psf[arm].load(os.path.join(PSF_PATH.format(arm), 'pca.h5'))

    template_psf[arm] = PcaPsf.from_psf(gauss_psf[arm], grid.wave, size=s, truncate=5)
    # print(grid.wave.shape, 
    #     template_psf[arm].wave.shape, template_psf[arm].dwave.shape, template_psf[arm].pc.shape)

## Generate downgraded template

In [0]:
def get_model():
    # model = grid.get_model(M_H=-0.5, T_eff=5500, log_g=4.5, a_M=0.0)
    model = grid.get_model(M_H=-0.5, T_eff=5500, log_g=4.5, a_M=0.0)
    model = grid.get_model(M_H=-1.5, T_eff=5000, log_g=3.0, a_M=0.0)
    model.convolve_psf(gauss_psf['mr'])
    return model

In [0]:
model = get_model()

f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)
ax.plot(model.wave, model.flux, lw=0.3)
ax.set_xlim(6000, 6050)

## Load observation

In [0]:
from pfs.ga.pfsspec.stellar import StellarSpectrum
from pfs.ga.pfsspec.sim.obsmod.calibration import FluxCalibrationBias

def get_spec(noise=1.0, calib_bias=None):
    #fn = '/datascope/subaru/user/dobos/data/pfsspec/snr/MW_disk_MS_G_0.0_5_0.5_mr_23.0.txt'
    fn = '/datascope/subaru/user/dobos/data/pfsspec/snr/dSph_RGB_0.0_5_0.5_mr_23.0.txt'
    df = pd.read_csv(fn, header=None, names=['wave', 'flux', 'flux_err'], delimiter='\s+')

    spec = StellarSpectrum()
    spec.wave = np.array(df['wave'])
    spec.flux = np.array(df['flux'])
    spec.flux_err = np.array(df['flux_err'])

    spec.flux = spec.flux + noise * spec.flux_err * np.random.normal(size=spec.wave.shape)

    if calib_bias:
        bias = FluxCalibrationBias(reuse_bias=True)
        bias.amplitude = calib_bias
        bias.apply_calibration(spec)
    else:
        bias = None

    return spec, bias

In [0]:
spec, bias = get_spec(calib_bias=0.05)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

ax.plot(spec.wave, spec.flux, lw=0.3)
ax.plot(spec.wave, spec.flux_err, lw=0.3)

In [0]:
from pfs.ga.pfsspec.core.obsmod.resampling import FluxConservingResampler

model = grid.get_model(M_H=-0.5, T_eff=5500, log_g=4.5, a_M=0.0)
model.convolve_psf(gauss_psf['mr'])
model.apply_resampler(FluxConservingResampler(), spec.wave, None)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

ax.plot(spec.wave, spec.flux, lw=0.3)
ax.plot(model.wave, model.flux * 0.45e-32, lw=0.3)

ax.set_xlim(8400, 8750)
ax.set_ylim(0, 0.5e-17)

# Continuum correction

In [0]:
npoly = 1
polys = np.ones((spec.wave.shape[0], npoly))

In [0]:
npoly = 10
normwave = (spec.wave - spec.wave[0]) / (spec.wave[-1] - spec.wave[0]) * 2 - 1       # -1..1
polys = np.zeros((spec.wave.shape[0], npoly))

coeffs = np.eye(npoly)
for i in range(npoly):
    polys[:, i] = np.polynomial.Chebyshev(coeffs[i])(normwave)
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

for i in range(npoly):
    ax.plot(spec.wave, polys[:, i], lw=0.3)

# Compare methods

In [0]:
spec, bias = get_spec(noise=1, calib_bias=0.05)

In [0]:
#zz = np.linspace(-0.00005, 0.00005, 201)
zz = np.linspace(-0.0005, 0.0005, 201)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

ax.plot(spec.wave, spec.flux, lw=0.3)

ax.set_xlim(8400, 8750)
ax.set_ylim(0, 0.5e-17)

## Alex

In [0]:
model = get_model()
nu_alex = np.empty_like(zz)

aa_alex = np.empty(zz.shape)
for i, z in enumerate(zz):
    m = type(model)(orig=model)
    m.wave = m.wave * (1 + z)
    m.apply_resampler(FluxConservingResampler(), spec.wave, None)

    phi = np.sum(m.flux * spec.flux / spec.flux_err ** 2)
    chi = np.sum(m.flux ** 2 / spec.flux_err ** 2)

    aa_alex[i] = phi / chi
    # print(aa[i])

    nu_alex[i] = phi / np.sqrt(chi)
    # print('nu', nu[i])

In [0]:
aa_alex[zz.shape[0] // 2], nu_alex[zz.shape[0] // 2]

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

ax.plot(zz * 3e5, nu_alex)
ax.grid()
ax.set_xlabel(r'$\Delta RV$')
ax.set_ylabel(r'$\nu$')
ax.set_title('Significance')

f.tight_layout()

## Alex with continuum correction

In [0]:
model = get_model()
nu_alex_cc = np.empty_like(zz)

aa_alex_cc = np.empty(zz.shape + (npoly,))
for i, z in enumerate(zz):
    m = type(model)(orig=model)
    m.wave = m.wave * (1 + z)
    m.apply_resampler(FluxConservingResampler(), spec.wave, None)

    phi = np.sum(spec.flux[:, None] * m.flux[:, None] * polys / spec.flux_err[:, None] ** 2, axis=0)
    # print('phi', phi.shape)

    chi = np.sum((m.flux[:, None, None] ** 2) * np.matmul(polys[:, :, None], polys[:, None, :]) / spec.flux_err[:, None, None] ** 2, axis=0)
    # print('chi', chi.shape)

    chi_inv = np.linalg.inv(chi)
    # print('chi_inv', chi_inv)

    aa_alex_cc[i, :] = chi_inv_phi = np.matmul(chi_inv, phi)
    # print('aa', aa[i, :])

    nu_alex_cc[i] = np.sqrt(np.dot(phi, chi_inv_phi))
    # print('nu', nu[i])

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

ax.plot(zz * 3e5, nu_alex_cc)
ax.grid()
ax.set_xlabel(r'$\Delta RV$')
ax.set_ylabel(r'$\nu$')
ax.set_title('Significance with flux correction')

f.tight_layout()

In [0]:
best = np.argmax(nu_alex_cc)

best, aa_alex_cc[best]

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

ax.plot(spec.wave, np.sum(aa_alex_cc[best] * polys, axis=-1) / aa_alex_cc[best, 0], lw=1,
    label='flux correction')
ax.plot(spec.wave, bias.bias, lw=1, label='simulated flux calibration error')

ax.set_ylim(0, 1.1)
ax.grid()

ax.set_title('Flux correction at best significance')

ax.legend()

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

m = type(model)(orig=model)
m.wave = m.wave * (1 + zz[best])
m.apply_resampler(FluxConservingResampler(), spec.wave, None)

ax.plot(spec.wave, spec.flux, lw=0.3)
ax.plot(spec.wave, m.flux * np.sum(aa_alex_cc[best] * polys, axis=-1), lw=0.3)

ax.set_xlim(8400, 8750)
ax.set_ylim(0, 0.3e-17)

## Koposov

In [0]:
import scipy.linalg

In [0]:
model = get_model()
chisq = np.empty_like(zz)

aa = np.empty(zz.shape + (npoly,))
for i, z in enumerate(zz):
    m = type(model)(orig=model)
    m.wave = m.wave * (1 + z)
    m.apply_resampler(FluxConservingResampler(), spec.wave, None)

    # Solve for poly coeffs
    normspec = spec.flux / spec.flux_err
    normtempl = m.flux / spec.flux_err
    normtempl /= np.median(normtempl)
    logl_z = np.log(spec.flux_err).sum()
    #print(logl_z)

    polys1 = normtempl[None, :] * polys.T
    vector1 = polys1 @ normspec
    matrix1 = np.dot(polys1, polys1.T)
    u, s, v = scipy.linalg.svd(matrix1, check_finite=False)
    #print('s', s)
    detI = np.prod(s)
    #print(detI)
    v2 = v.T @ ((1. / s)[:, None] * u.T) @ vector1  # this is matrix1^(-1) vector1

    aa[i, :] = chi_inv_phi = np.matmul(chi_inv, phi)

    chisq[i] = -vector1.T @ v2 - np.log(1. / detI) + 2 * logl_z + np.dot(
        normspec, normspec)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

ax.plot(zz * 3e5, chisq)
ax.grid()

ax.set_xlabel(r'$\Delta RV$')
ax.set_ylabel(r'$\chi^2$')
ax.set_title(r'$\chi^2$ with flux correction (Koposov)')

f.tight_layout()

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

ax.plot(zz * 3e5, nu_alex / np.max(nu_alex), lw=0.5, label='Significance')
ax.plot(zz * 3e5, nu_alex_cc / np.max(nu_alex_cc), lw=0.5, label='Significance with flux correction')

ax.plot(zz * 3e5, -chisq / np.max(-chisq), lw=0.5, label=r'$\chi^2$ with flux correction (Koposov)')

ax.grid()

ax.set_xlabel(r'$\Delta RV$')
ax.legend()

ax.set_title('SNR={:.2f}'.format(np.quantile(spec.flux / spec.flux_err, 0.95)))